In [1]:
import numpy as np
import agentpy as ap
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
import IPython


# TODO : generar en diferentes tiempos los carros

class Vehicle(ap.Agent):
    # Corregir el metodo de movement para que el carro deje de moverse solamente cuando este adyacente al semaforo y este esté en rojo.
    # Agregar que el carro se pare cuando tiene un carro en frente parado
    def setup(self):
        self.grid = self.model.grid
        self.pos = [0, 0]
        self.road = 1
        self.side = [1, 0]
        self.speed = 1
        self.crossed = False

    def direction(self):
        self.pos = self.grid.positions[self]
        if self.pos[1] == 0:
            self.side = [0, 1]

    def movement(self):
        self.direction()

        return (self.speed * self.side[0], self.speed * self.side[1])
    def route(self):
        self.pos = self.grid.positions[self]
        if self.pos[1] == 0:
            return 'HORIZONTAL'
        return 'VERTICAL'

class StopSign(ap.Agent):
     # TODO: Agregar metodo que calcule el cambio de estados del semaforo dependiendo de la cantidad de carros.
    def setup(self):
        self.status = 1
        self.road = 3
        self.grid = self.model.grid
        self.pos = [0, 0]
        self.route=''

    def positions(self):
        self.pos = self.grid.positions[self]

    def state(self):
        self.positions()
        tGrid = self.p['Grid']
        self.status = 2
        self.road=2
        if self.model.n_cars_1 > self.model.n_cars_2:
            if self.pos[0] == int((tGrid/2)+1):
                self.status = 0
                self.road=4
        elif self.model.n_cars_1 < self.model.n_cars_2:
            if self.pos[1] == int((tGrid/2)+1):
                self.status = 0
                self.road=4
        else:
            if self.pos[1] == int((tGrid/2)+1):
                self.status = 0
                self.road=4


class Roads(ap.Agent):
    def setup(self):
        self.road = 1
        self.condition = 1


class Intersection(ap.Model):
    def setup(self):
        self.con=0
        # Grid
        tGrid = self.p['Grid']
        self.grid = ap.Grid(self, [tGrid] * 2, torus=True,track_empty=True)

        self.n_cars_1 = 0
        self.n_cars_2 = 0

        # Agentes
        n_vehicles = self.p['Vehicles']
        n_roads = tGrid*2

        self.vehicles = ap.AgentList(self, n_vehicles, Vehicle)

        self.road = ap.AgentList(self, n_roads, Roads)
        self.stop_sign = ap.AgentList(self, 2, StopSign)

        self.vehicles.grid = self.grid

        road_pos = []
        tGrid = self.p['Grid']
        for i in range(tGrid*2):
            if i <= tGrid-1:
                road_pos.append((i, int(tGrid/2)))
            else:
                road_pos.append((int(tGrid/2), i - tGrid))
            
        self.grid.add_agents(self.stop_sign, positions=[(int((tGrid/2)-1), int((tGrid/2)+1)), (int((tGrid/2)+1), int((tGrid/2)-1))])
        stop_light = self.stop_sign
        for semaforo in stop_light:
            if self.grid.positions[semaforo][1]==0:
                semaforo.route='VERTICAL'
            else: semaforo.route='HORIZONTAL'
        vehicles_positions=[]
        for i in range (1,n_vehicles+1):
            if i%2==0:
                vehicles_positions.append((int(tGrid/2), 0))
            else:
                vehicles_positions.append((0,int(tGrid/2)))

        
    def step(self):
        # TODO: agregar logica para la interaccion entre el objeto semaforo y objeto vehiculo
        tGrid = self.p['Grid']
        n_vehicles = self.p['Vehicles']
        if self.con==0:
            vehicles_positions=[]
            positions=[(0, int(tGrid/2)), (int(tGrid/2), 0)]
            
            for i in range (n_vehicles):
                rand_pos = np.random.randint(0, 2)
                vehicles_positions.append(positions[rand_pos])
           
            new_car=ap.AgentList(self,1,Vehicle)
            self.grid.add_agents(self.vehicles,vehicles_positions)
            #Posicion randas
            self.con+=1

        n_movements=0
        move_cars=0
        movimiento=True
        state=False
        moving_cars_1 = self.vehicles

        
        for car in moving_cars_1:
            agent_pos=self.grid.positions[car]
            for i in range(1,int((tGrid/2))):
                if (int(tGrid/2) == agent_pos[0])and(int(i) == agent_pos[1]):
                    self.n_cars_1 += 1
                    state=True
                if  (int(tGrid/2) == agent_pos[1])and(int(i) == agent_pos[0]):
                    self.n_cars_2 += 1
                    state=True
        if state:
            stop_light = self.stop_sign
            for semaforo in stop_light:    
                if state:
                    semaforo.state()
                else:
                    semaforo.status=1
                    semaforo.road=3

        
       
        for agent in self.grid.agents:
            agent_pos=self.grid.positions[agent]
            movimiento=True
            if agent.type == 'Vehicle':
                for neighbor in self.grid.neighbors(agent):
                    if agent.route() == 'VERTICAL':
                        if self.grid.positions[neighbor][1]==agent_pos[1]+1 and self.grid.positions[neighbor][0]==agent_pos[0]:
                            if   neighbor.type=='Vehicle':    
                                movimiento=False
                                break
                        if self.grid.positions[neighbor][1]==agent_pos[1] and self.grid.positions[neighbor][0]==agent_pos[0]+1:
                            if  neighbor.type=='StopSign':
                                if neighbor.status==2:
                                    movimiento=False
                                    break
                                else:
                                    self.n_cars_1-=1
                                    break
                        
                    if agent.route() == 'VERTICAL':
                        if self.grid.positions[neighbor][0]==agent_pos[0]+1 and self.grid.positions[neighbor][1]==agent_pos[1]:
                            if  neighbor.type=='Vehicle':
                                movimiento=False
                                break
                        if self.grid.positions[neighbor][0]==agent_pos[0] and self.grid.positions[neighbor][1]==agent_pos[1]+1:
                            if  neighbor.type=='StopSign':
                                if neighbor.status==2:
                                    movimiento=False
                                    break
                                else:
                                    self.n_cars_2-=1
                                    break
                if movimiento:
                    coordinates_move=agent.movement()
                    self.grid.move_by(agent,coordinates_move)

    def end(self):
        pass


parameters = {
    'Vehicles': 20,
    'steps': 500,
    'Grid':25,
}


def animation_plot(model, ax):
    attr_grid = model.grid.attr_grid('road')
    #0 es verde claro, 1 es gris, 2 es rojo, 3 amarillo, 4 verde fuerte
    color_dict = {0: '#d5e5d5', 1: '#e5e5e5', 2: '#d65c2c', 3: '#FFFF00', 4: '#21d41e', None: '#c3c3c3'}  # '#d5e5d5' '#d62c2c'
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)


fig, ax = plt.subplots()
model = Intersection(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=30))